In [1]:
with open("C:\\Users\\adams\\Documents\\Building LLM from scratch\\Data\\J. K. Rowling - Harry Potter 1 - Sorcerer's Stone.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [2]:
len(raw_text)

439742

In [3]:
print(raw_text[:99])

Harry Potter and the Sorcerer's Stone


CHAPTER ONE

THE BOY WHO LIVED

Mr. and Mrs. Dursley, of nu


I am using Harry potter chapter one book

In [4]:
import os

data_dir = "Data"
book_files = sorted([f for f in os.listdir(data_dir) if f.endswith(".txt")])

combined_text = ""
for file in book_files:
    file_path = os.path.join(data_dir, file)
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            combined_text += f.read() + "\n"
    except UnicodeDecodeError:
        with open(file_path, "r", encoding="ISO-8859-1") as f:
            combined_text += f.read() + "\n"


In [5]:
combined_text[:1000]

"Harry Potter and the Sorcerer's Stone\n\n\nCHAPTER ONE\n\nTHE BOY WHO LIVED\n\nMr. and Mrs. Dursley, of number four, Privet Drive, were proud to say\nthat they were perfectly normal, thank you very much. They were the last\npeople you'd expect to be involved in anything strange or mysterious,\nbecause they just didn't hold with such nonsense.\n\nMr. Dursley was the director of a firm called Grunnings, which made\ndrills. He was a big, beefy man with hardly any neck, although he did\nhave a very large mustache. Mrs. Dursley was thin and blonde and had\nnearly twice the usual amount of neck, which came in very useful as she\nspent so much of her time craning over garden fences, spying on the\nneighbors. The Dursleys had a small son called Dudley and in their\nopinion there was no finer boy anywhere.\n\nThe Dursleys had everything they wanted, but they also had a secret, and\ntheir greatest fear was that somebody would discover it. They didn't\nthink they could bear it if anyone found ou

In [6]:
print("Total characters:", len(combined_text))
print("Unique characters:", len(set(combined_text)))
print("First 200 characters preview:\n", combined_text[:200])

Total characters: 2652656
Unique characters: 91
First 200 characters preview:
 Harry Potter and the Sorcerer's Stone


CHAPTER ONE

THE BOY WHO LIVED

Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say
that they were perfectly normal, thank you very much. They


In [9]:
import re

In [10]:
text = "Hello , world. I am Harry Potter"

result = re.split(r'(\s)', text)

print(result)

['Hello', ' ', ',', ' ', 'world.', ' ', 'I', ' ', 'am', ' ', 'Harry', ' ', 'Potter']


In [11]:
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world.', 'I', 'am', 'Harry', 'Potter']


In [30]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s+)', combined_text)

preprocessed = [item.strip() for item in preprocessed if item.strip()]

In [31]:
print(preprocessed[:30])

['Harry', 'Potter', 'and', 'the', 'Sorcerer', "'", 's', 'Stone', 'CHAPTER', 'ONE', 'THE', 'BOY', 'WHO', 'LIVED', 'Mr', '.', 'and', 'Mrs', '.', 'Dursley', ',', 'of', 'number', 'four', ',', 'Privet', 'Drive', ',', 'were', 'proud']


In [32]:
print(len(preprocessed))

607954


create token IDs

In [33]:
all_words = sorted(set(preprocessed))

In [34]:
vocab_size = len(all_words)
print(vocab_size)

18609


In [35]:
vocab = {token:integer for integer,token in enumerate(all_words)}

In [36]:
for i,item in enumerate(vocab.items()):
    print(item)
    if i>10:
        break

('!', 0)
('"', 1)
('$', 2)
('%', 3)
("'", 4)
('(', 5)
(')', 6)
('*', 7)
('*$32*', 8)
('*%$4*', 9)
('*%08*', 10)
('*%16*', 11)


In [37]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [38]:
all_words.extend(["<|endoftext|>", "<|unk|>"])


In [39]:
vocab_size = len(all_words)
print(vocab_size)

18611


In [40]:
vocab = {token:integer for integer,token in enumerate(all_words)}

In [43]:
tokenizer = SimpleTokenizerV2(vocab)

In [44]:
tokenizer.encode(text)

[2089, 138, 18448, 192, 2212, 5017, 2048, 3262]

In [45]:
tokenizer.decode(tokenizer.encode(text))

'Hello, world. I am Harry Potter'

In [46]:
!pip install tiktoken

In [47]:
import importlib
import tiktoken

In [48]:
tokenizer = tiktoken.get_encoding("gpt2")

In [49]:
enc_text = tokenizer.encode(raw_text)

In [50]:
print(len(enc_text))

116725


In [51]:
enc_sample = enc_text[:50]

In [52]:
print(enc_sample)

[18308, 14179, 290, 262, 30467, 338, 8026, 628, 198, 41481, 16329, 198, 198, 10970, 16494, 56, 19494, 406, 3824, 1961, 198, 198, 5246, 13, 290, 9074, 13, 360, 1834, 1636, 11, 286, 1271, 1440, 11, 4389, 16809, 9974, 11, 547, 6613, 284, 910, 198, 5562, 484, 547, 7138, 3487, 11]


In [53]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

In [54]:
import torch

In [56]:
from torch.utils.data import Dataset

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1:i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk, dtype=torch.long))
            self.target_ids.append(torch.tensor(target_chunk, dtype=torch.long))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


In [66]:
txt=combined_text

In [67]:
def create_dataloader_v1(
    txt,
    tokenizer=None,
    batch_size=4,
    max_length=256,
    stride=128,
    shuffle=True,
    drop_last=True,
    num_workers=0
):
    if tokenizer is None:
        tokenizer = tiktoken.get_encoding("gpt2")

    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )


In [69]:
print("PyTorch version:", torch.__version__)
dataloader = create_dataloader_v1(
    combined_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

PyTorch version: 2.4.0
[tensor([[18308, 14179,   290,   262]]), tensor([[14179,   290,   262, 30467]])]


Token Embeddings

In [70]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [71]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [72]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[18308, 14179,   290,   262],
        [30467,   338,  8026,   628],
        [  198, 41481, 16329,   198],
        [  198, 10970, 16494,    56],
        [19494,   406,  3824,  1961],
        [  198,   198,  5246,    13],
        [  290,  9074,    13,   360],
        [ 1834,  1636,    11,   286]])

Inputs shape:
 torch.Size([8, 4])


In [73]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [74]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [75]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [76]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])


implementing gpt architecture to generate text

In [77]:
import torch
import torch.nn as nn


class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )

        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, in_idx):
        bsz, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_ids = torch.arange(seq_len, device=in_idx.device)
        pos_embeds = self.pos_emb(pos_ids)
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits


class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"]
        )
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)
        x = self.drop_shortcut(x)
        x = x + shortcut

        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut

        return x


class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads

        self.W_q = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_k = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_v = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)

        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1).bool())

    def forward(self, x):
        B, T, D = x.size()

        q = self.W_q(x).view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        k = self.W_k(x).view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        v = self.W_v(x).view(B, T, self.num_heads, self.head_dim).transpose(1, 2)

        att = (q @ k.transpose(-2, -1)) / (self.head_dim ** 0.5)
        att = att.masked_fill(self.mask[:T, :T], float("-inf"))
        att = torch.softmax(att, dim=-1)
        att = self.dropout(att)

        out = (att @ v).transpose(1, 2).contiguous().view(B, T, self.d_out)
        return self.out_proj(out)


class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"])
        )

    def forward(self, x):
        return self.net(x)


class LayerNorm(nn.Module):
    def __init__(self, dim, eps=1e-5):
        super().__init__()
        self.eps = eps
        self.scale = nn.Parameter(torch.ones(dim))
        self.shift = nn.Parameter(torch.zeros(dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        return self.scale * (x - mean) / torch.sqrt(var + self.eps) + self.shift


class GELU(nn.Module):
    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(torch.sqrt(torch.tensor(2.0 / torch.pi)) * (x + 0.044715 * x ** 3)))


In [79]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,       # GPT-2 tokenizer vocab size
    "context_length": 1024,    # max sequence length
    "emb_dim": 768,            # embedding size
    "n_heads": 12,             # number of attention heads
    "n_layers": 12,            # transformer layers
    "drop_rate": 0.1,          # dropout rate
    "qkv_bias": False          # whether to use bias in QKV projection
}


In [80]:
from torch.optim import AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = GPTModel(GPT_CONFIG_124M).to(device)

optimizer = AdamW(model.parameters(), lr=3e-4)
loss_fn = nn.CrossEntropyLoss()


In [94]:
from tqdm import tqdm

epochs = 25
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for inputs, targets in tqdm(dataloader):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        logits = model(inputs)  # [B, T, V]
        loss = loss_fn(logits.view(-1, logits.size(-1)), targets.view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{epochs} | Loss: {avg_loss:.4f}")


  0%|          | 0/3647 [00:00<?, ?it/s]

100%|██████████| 3647/3647 [06:19<00:00,  9.62it/s]


Epoch 1/25 | Loss: 4.4372


100%|██████████| 3647/3647 [06:42<00:00,  9.05it/s]


Epoch 2/25 | Loss: 4.2487


...

Epoch 23/25 | Loss: 2.6329


100%|██████████| 3647/3647 [06:45<00:00,  9.00it/s]


Epoch 24/25 | Loss: 2.5441


100%|██████████| 3647/3647 [06:45<00:00,  9.00it/s]

Epoch 25/25 | Loss: 2.4766


Need more computation power to train the model until convergence.

In [95]:
def generate_text(model, tokenizer, context, max_new_tokens=50):
    model.eval()
    tokens = tokenizer.encode(context)[-GPT_CONFIG_124M["context_length"]:]
    x = torch.tensor(tokens, dtype=torch.long, device=device).unsqueeze(0)

    for _ in range(max_new_tokens):
        with torch.no_grad():
            logits = model(x)
        next_token = torch.argmax(logits[0, -1], dim=-1).unsqueeze(0)
        x = torch.cat([x, next_token.unsqueeze(0)], dim=1)
        x = x[:, -GPT_CONFIG_124M["context_length"]:]

    return tokenizer.decode(x[0].tolist())


In [96]:
import tiktoken
enc = tiktoken.get_encoding("gpt2")
print(generate_text(model, enc, "Harry turned around and saw", max_new_tokens=100))


Harry turned around and saw, the

 the the, the down down down, the, the,
 down down Gry's,
sh's, Hermione, Hermione, the the
 the the stone, Hermione, the
sh Gry's, Hermione, Gry, the Gry,, and pointed, Hermione, and was pointed Gry and pointed and pointed, and pointed, the Gry and Hermione, the and Hermione,
sh the Gry, and the Gry, Gry, and pointed Gry, Gry, Hermione, Hermione,


In [97]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params:,}")

Total number of parameters: 163,009,536


In [98]:
print("Token embedding layer shape:", model.tok_emb.weight.shape)
print("Output layer shape:", model.out_head.weight.shape)

Token embedding layer shape: torch.Size([50257, 768])
Output layer shape: torch.Size([50257, 768])


In [99]:
total_params_gpt2 = total_params - sum(p.numel() for p in model.out_head.parameters())
print(f"Number of trainable parameters considering weight tying: {total_params_gpt2:,}")

Number of trainable parameters considering weight tying: 124,412,160


In [100]:
total_size_bytes = total_params * 4 #A
total_size_mb = total_size_bytes / (1024 * 1024) #B
print(f"Total size of the model: {total_size_mb:.2f} MB")

Total size of the model: 621.83 MB


In [101]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    # idx is (batch, n_tokens) array of indices in the current context
    for _ in range(max_new_tokens):
        
        # Crop current context if it exceeds the supported context size
        # E.g., if LLM supports only 5 tokens, and the context size is 10
        # then only the last 5 tokens are used as context
        idx_cond = idx[:, -context_size:]
        
        # Get the predictions
        with torch.no_grad():
            logits = model(idx_cond)
        
        # Focus only on the last time step
        # (batch, n_tokens, vocab_size) becomes (batch, vocab_size)
        logits = logits[:, -1, :]  

        # Apply softmax to get probabilities
        probas = torch.softmax(logits, dim=-1)  # (batch, vocab_size)

        # Get the idx of the vocab entry with the highest probability value
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)  # (batch, 1)

        # Append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)

    return idx

In [102]:
start_context = "my school was"
encoded = tokenizer.encode(start_context)
print("encoded:", encoded)
encoded_tensor = torch.tensor(encoded).unsqueeze(0) #A
print("encoded_tensor.shape:", encoded_tensor.shape)

encoded: [1820, 1524, 373]
encoded_tensor.shape: torch.Size([1, 3])
